In [351]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import re

In [352]:
def convert_google_sheet_url(url):
    # regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'
    # replace function to construct the new URL for CSV export
    # if gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'
    # replace using regex
    new_url = re.sub(pattern, replacement, url)
    return new_url

game_data_2024_url = 'https://docs.google.com/spreadsheets/d/1F8GBPtLhugdO0pqrJe3O9fMPOYv3AqVRq8ouUZWZxIY/edit#gid=0'
new_game_data_2024_url = convert_google_sheet_url(game_data_2024_url)
game_data_2024 = pd.read_csv(new_game_data_2024_url)

game_data_2023_url = 'https://docs.google.com/spreadsheets/d/1F8GBPtLhugdO0pqrJe3O9fMPOYv3AqVRq8ouUZWZxIY/edit#gid=256452955'
new_game_data_2023_url = convert_google_sheet_url(game_data_2023_url)
game_data_2023 = pd.read_csv(new_game_data_2023_url)

In [353]:
game_data = pd.concat([game_data_2024, game_data_2023], ignore_index=True)

In [354]:
game_data['batter_team'] = game_data['away_team']
game_data.loc[game_data['inning_topbot'] == 'Bot', 'batter_team'] = game_data.loc[game_data['inning_topbot'] == 'Bot', 'home_team']

game_data['pitcher_team'] = game_data['home_team']
game_data.loc[game_data['inning_topbot'] == 'Bot', 'pitcher_team'] = game_data.loc[game_data['inning_topbot'] == 'Bot', 'away_team']

In [355]:
game_data['balls'] = game_data['balls'].fillna(0)
game_data['strikes'] = game_data['strikes'].fillna(0)
game_data['count'] = game_data['balls'].astype(int).astype(str) + '-' + game_data['strikes'].astype(int).astype(str)

In [356]:
game_data['count_advantage'] = 'neutral'
game_data.loc[(game_data['count'] == '1-0') | (game_data['count'] == '2-0') | (game_data['count'] == '2-1') | (game_data['count'] == '3-0') | (game_data['count'] == '3-1'), 'count_advantage'] = 'batter'
game_data.loc[(game_data['count'] == '0-1') | (game_data['count'] == '0-2') | (game_data['count'] == '1-2'), 'count_advantage'] = 'pitcher'

In [357]:
game_data['CH'] = game_data['pitch_type'] == 'CH'
game_data['CH'] = game_data['CH'].replace({True: 1, False: 0})

game_data['CU'] = game_data['pitch_type'] == 'CU'
game_data['CU'] = game_data['CU'].replace({True: 1, False: 0})

game_data['SL'] = game_data['pitch_type'] == 'SL'
game_data['SL'] = game_data['SL'].replace({True: 1, False: 0})

game_data['FF'] = game_data['pitch_type'] == 'FF'
game_data['FF'] = game_data['FF'].replace({True: 1, False: 0})

game_data['SI'] = game_data['pitch_type'] == 'SI'
game_data['SI'] = game_data['SI'].replace({True: 1, False: 0})

game_data['CR'] = game_data['pitch_type'] == 'CR'
game_data['CR'] = game_data['CR'].replace({True: 1, False: 0})

/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3044767723.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['CH'] = game_data['CH'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3044767723.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['CU'] = game_data['CU'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3044767723.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will 

In [358]:
# game_data['FsB'] = game_data['pitch_type'].isin(['FF', 'FT', 'SI', 'FC'])
# game_data['FsB'] = game_data['FsB'].replace({True: 1, False: 0})

# game_data['OfS'] = game_data['pitch_type'].isin(['CH', 'CR', 'FS', 'FO'])
# game_data['OfS'] = game_data['OfS'].replace({True: 1, False: 0})

# game_data['BrB'] = game_data['pitch_type'].isin(['CU', 'SL', 'KC', 'ST', 'SV'])
# game_data['BrB'] = game_data['BrB'].replace({True: 1, False: 0})

# game_data['OtR'] = game_data['pitch_type'].isin(['PO', 'KN', 'SC'])
# game_data['OtR'] = game_data['OtR'].replace({True: 1, False: 0})

game_data['pitch_category'] = 'other'
game_data.loc[game_data['pitch_type'].isin(['FF', 'FT', 'SI', 'FC']), 'pitch_category'] = 'fastball'
game_data.loc[game_data['pitch_type'].isin(['CH', 'CR', 'FS', 'FO']), 'pitch_category'] = 'offspeed'
game_data.loc[game_data['pitch_type'].isin(['CU', 'SL', 'KC', 'ST', 'SV']), 'pitch_category'] = 'breaking ball'

In [359]:
game_data['spray_angle'] = np.nan
# use np.arctan2 then convert to degrees
game_data.loc[(game_data['hc_x'].isna() == False) & (game_data['hc_y'].isna() == False), 'spray_angle'] = np.arctan2(game_data['hc_y'], game_data['hc_x']) * 180 / np.pi

In [360]:
game_data['hit_field_direction'] = None
game_data.loc[(game_data['spray_angle'] >= 105) | (game_data['spray_angle'] <= -90), 'hit_field_direction'] = 'left'
game_data.loc[(game_data['spray_angle'] < 105) & (game_data['spray_angle'] > 75), 'hit_field_direction'] = 'center'
game_data.loc[(game_data['spray_angle'] <= 75) | ((game_data['spray_angle'] < 0) & (game_data['spray_angle'] > -90)), 'hit_field_direction'] = 'right'

In [361]:
game_data['hit_direction'] = None
game_data.loc[game_data['hit_field_direction'] == 'center', 'hit_direction'] = 'cent'
game_data.loc[((game_data['stand'] == 'R') & (game_data['hit_field_direction'] == 'left')) | ((game_data['stand'] == 'L') & (game_data['hit_field_direction'] == 'right')), 'hit_direction'] = 'pull'
game_data.loc[((game_data['stand'] == 'L') & (game_data['hit_field_direction'] == 'left')) | ((game_data['stand'] == 'R') & (game_data['hit_field_direction'] == 'right')), 'hit_direction'] = 'oppo'

In [362]:
game_data['BIP_hit_direction'] = game_data['hit_direction'].isna() == False
game_data['BIP_hit_direction'] = game_data['BIP_hit_direction'].replace({True: 1, False: 0})

game_data['Pull'] = game_data['hit_direction'] == 'pull'
game_data['Pull'] = game_data['Pull'].replace({True: 1, False: 0})

game_data['Cent'] = game_data['hit_direction'] == 'cent'
game_data['Cent'] = game_data['Cent'].replace({True: 1, False: 0})

game_data['Oppo'] = game_data['hit_direction'] == 'oppo'
game_data['Oppo'] = game_data['Oppo'].replace({True: 1, False: 0})

/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3244586847.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['BIP_hit_direction'] = game_data['BIP_hit_direction'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3244586847.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['Pull'] = game_data['Pull'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3244586847.py:8: FutureWarning: Downcasting behavior in

In [363]:
game_data['BIP'] = game_data['type'] == 'X'
game_data['BIP'] = game_data['BIP'].replace({True: 1, False: 0})

/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/572917243.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['BIP'] = game_data['BIP'].replace({True: 1, False: 0})


In [364]:
game_data['BIP_bb_type'] = game_data['bb_type'].isna() == False
game_data['BIP_bb_type'].replace({True: 1, False: 0})

game_data['LD'] = game_data['bb_type'] == 'line_drive'
game_data['LD'] = game_data['LD'].replace({True: 1, False: 0})

game_data['GB'] = game_data['bb_type'].isin(['bunt', 'ground_ball'])
game_data['GB'] = game_data['GB'].replace({True: 1, False: 0})

game_data['FB'] = game_data['bb_type'].isin(['fly_ball', 'popup'])
game_data['FB'] = game_data['FB'].replace({True: 1, False: 0})

/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/683621109.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['BIP_bb_type'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/683621109.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['LD'] = game_data['LD'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/683621109.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed i

In [365]:
game_data['Pitches'] = game_data['description'].isna() == False
game_data['Pitches'] = game_data['Pitches'].replace({True: 1, False: 0})

game_data['Zone'] = (game_data['plate_x'] >= -9.97/12) & (game_data['plate_x'] <= 9.97/12) & (game_data['plate_z'] >= 18.29/12) & (game_data['plate_z'] <= 44.08/12)
game_data['Zone'] = game_data['Zone'].replace({True: 1, False: 0})

game_data['Swing'] = game_data['description'].isin(['bunt_foul_tip', 'foul', 'foul_bunt', 'hit_into_play', 'missed_bunt', 'swinging_strike', 'swinging_strike_blocked'])
game_data['Swing'] = game_data['Swing'].replace({True: 1, False: 0})

game_data['Contact'] = game_data['description'].isin(['bunt_foul_tip', 'foul', 'foul_bunt', 'hit_into_play'])
game_data['Contact'] = game_data['Contact'].replace({True: 1, False: 0})

game_data['Z_Swing'] = (game_data['Zone'] == 1) & (game_data['Swing'] == 1)
game_data['Z_Swing'] = game_data['Z_Swing'].replace({True: 1, False: 0})

game_data['O_Swing'] = (game_data['Zone'] == 0) & (game_data['Swing'] == 1)
game_data['O_Swing'] = game_data['O_Swing'].replace({True: 1, False: 0})

game_data['Z_Contact'] = (game_data['Z_Swing'] == 1) & (game_data['Contact'] == 1)
game_data['Z_Contact'] = game_data['Z_Contact'].replace({True: 1, False: 0})

game_data['O_Contact'] = (game_data['O_Swing'] == 1) & (game_data['Contact'] == 1)
game_data['O_Contact'] = game_data['O_Contact'].replace({True: 1, False: 0})

/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3006474286.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['Pitches'] = game_data['Pitches'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3006474286.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game_data['Zone'] = game_data['Zone'].replace({True: 1, False: 0})
/var/folders/8k/8dtwqkz13n7_yyj509ckx33w0000gn/T/ipykernel_91668/3006474286.py:8: FutureWarning: Downcasting behavior in `replace` is deprec

In [366]:
game_data['Hard_Hit'] = (game_data['launch_speed_angle'] == 5) | (game_data['launch_speed_angle'] == 6)

In [367]:
team = 'HAM'
batter = 'Luis Bernardo'

team_batter_data = game_data[game_data['batter_team'] == team]

In [368]:
team_batter_totals = team_batter_data.groupby('batter').agg(
    BIP=pd.NamedAgg(column='BIP', aggfunc='sum'),
    BIP_hit_direction=pd.NamedAgg(column='BIP_hit_direction', aggfunc='sum'),
    Pull=pd.NamedAgg(column='Pull', aggfunc='sum'),
    Cent=pd.NamedAgg(column='Cent', aggfunc='sum'),
    Oppo=pd.NamedAgg(column='Oppo', aggfunc='sum'),
    BIP_bb_type=pd.NamedAgg(column='BIP_bb_type', aggfunc='sum'),
    LD=pd.NamedAgg(column='LD', aggfunc='sum'),
    GB=pd.NamedAgg(column='GB', aggfunc='sum'),
    FB=pd.NamedAgg(column='FB', aggfunc='sum'),
    Pitches=pd.NamedAgg(column='Pitches', aggfunc='sum'),
    Zone=pd.NamedAgg(column='Zone', aggfunc='sum'),
    Swing=pd.NamedAgg(column='Swing', aggfunc='sum'),
    Contact=pd.NamedAgg(column='Contact', aggfunc='sum'),
    Z_Swing=pd.NamedAgg(column='Z_Swing', aggfunc='sum'),
    O_Swing=pd.NamedAgg(column='O_Swing', aggfunc='sum'),
    Z_Contact=pd.NamedAgg(column='Z_Contact', aggfunc='sum'),
    O_Contact=pd.NamedAgg(column='O_Contact', aggfunc='sum'),
    Hard_Hit=pd.NamedAgg(column='Hard_Hit', aggfunc='sum')
)

In [369]:
team_batter_rates = team_batter_totals

team_batter_rates['Pull_pct'] = team_batter_rates['Pull']/team_batter_rates['BIP_hit_direction']
team_batter_rates['Cent_pct'] = team_batter_rates['Cent']/team_batter_rates['BIP_hit_direction']
team_batter_rates['Oppo_pct'] = team_batter_rates['Oppo']/team_batter_rates['BIP_hit_direction']

team_batter_rates['LD_pct'] = team_batter_rates['LD']/team_batter_rates['BIP_bb_type']
team_batter_rates['GB_pct'] = team_batter_rates['GB']/team_batter_rates['BIP_bb_type']
team_batter_rates['FB_pct'] = team_batter_rates['FB']/team_batter_rates['BIP_bb_type']

team_batter_rates['Swing_pct'] = team_batter_rates['Swing']/team_batter_rates['Pitches']
team_batter_rates['Contact_pct'] = team_batter_rates['Contact']/team_batter_rates['Swing']

team_batter_rates['Z_Swing_pct'] = team_batter_rates['Z_Swing']/team_batter_rates['Zone']
team_batter_rates['O_Swing_pct'] = team_batter_rates['O_Swing']/team_batter_rates['Zone']

team_batter_rates['Z_Contact_pct'] = team_batter_rates['Z_Contact']/team_batter_rates['Zone']
team_batter_rates['O_Contact_pct'] = team_batter_rates['O_Contact']/team_batter_rates['Zone']

team_batter_rates['Hard_Hit_pct'] = team_batter_rates['Hard_Hit']/team_batter_rates['BIP']

team_batter_rates = team_batter_rates[['BIP', 'BIP_hit_direction', 'Pull_pct', 'Cent_pct', 'Oppo_pct', 
                   'BIP_bb_type', 'LD_pct', 'GB_pct', 'FB_pct', 
                   'Pitches', 'Swing_pct', 'Contact_pct',
                   'Z_Swing_pct', 'O_Swing_pct', 'Z_Contact_pct', 'O_Contact_pct', 
                   'Hard_Hit_pct']]

team_batter_rates.round(3)

,BIP,BIP_hit_direction,Pull_pct,Cent_pct,Oppo_pct,BIP_bb_type,LD_pct,GB_pct,FB_pct,Pitches,Swing_pct,Contact_pct,Z_Swing_pct,O_Swing_pct,Z_Contact_pct,O_Contact_pct,Hard_Hit_pct
batter,,,,,,,,,,,,,,,,,
Aidan Muir,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,3,1.000,0.333,1.000,0.500,0.000,0.500,NaN
Brandon Latter,1,1,0.000,0.000,1.000,1,1.000,0.000,0.000,6,0.333,1.000,0.667,0.000,0.667,0.000,1.000
Brandon Nicholson,21,17,0.353,0.529,0.118,17,0.176,0.412,0.412,117,0.547,0.750,0.783,2.000,0.609,1.478,0.429
Brendon Daley,10,10,0.500,0.200,0.300,10,0.300,0.600,0.100,46,0.500,0.652,0.900,1.400,0.800,0.700,0.300
Bryan Rivera,5,5,0.000,0.600,0.400,5,0.200,0.600,0.200,45,0.600,0.815,0.938,0.750,0.875,0.500,0.200
Charlie Towers,10,10,0.400,0.200,0.400,10,0.200,0.500,0.300,40,0.550,0.636,0.842,0.316,0.632,0.105,0.200
Clayton Keyes,9,9,0.222,0.222,0.556,9,0.222,0.444,0.333,92,0.359,0.576,0.652,0.783,0.435,0.391,0.333
Connor Bowie,6,6,0.500,0.167,0.333,6,0.000,0.500,0.500,19,0.526,0.800,1.000,4.000,1.000,3.000,0.333
Danny Berg,14,11,0.545,0.273,0.182,11,0.182,0.545,0.273,79,0.380,0.800,0.682,0.682,0.636,0.455,0.357


In [370]:
team_batter_rates[team_batter_rates.index == batter].round(3)

,BIP,BIP_hit_direction,Pull_pct,Cent_pct,Oppo_pct,BIP_bb_type,LD_pct,GB_pct,FB_pct,Pitches,Swing_pct,Contact_pct,Z_Swing_pct,O_Swing_pct,Z_Contact_pct,O_Contact_pct,Hard_Hit_pct
batter,,,,,,,,,,,,,,,,,
Luis Bernardo,39,36,0.444,0.222,0.333,36,0.139,0.361,0.5,138,0.536,0.824,0.833,1.222,0.722,0.972,0.359


In [371]:
team_batter_totals_p_throws = team_batter_data.groupby(['batter', 'p_throws']).agg(
    BIP=pd.NamedAgg(column='BIP', aggfunc='sum'),
    BIP_hit_direction=pd.NamedAgg(column='BIP_hit_direction', aggfunc='sum'),
    Pull=pd.NamedAgg(column='Pull', aggfunc='sum'),
    Cent=pd.NamedAgg(column='Cent', aggfunc='sum'),
    Oppo=pd.NamedAgg(column='Oppo', aggfunc='sum'),
    BIP_bb_type=pd.NamedAgg(column='BIP_bb_type', aggfunc='sum'),
    LD=pd.NamedAgg(column='LD', aggfunc='sum'),
    GB=pd.NamedAgg(column='GB', aggfunc='sum'),
    FB=pd.NamedAgg(column='FB', aggfunc='sum'),
    Pitches=pd.NamedAgg(column='Pitches', aggfunc='sum'),
    Zone=pd.NamedAgg(column='Zone', aggfunc='sum'),
    Swing=pd.NamedAgg(column='Swing', aggfunc='sum'),
    Contact=pd.NamedAgg(column='Contact', aggfunc='sum'),
    Z_Swing=pd.NamedAgg(column='Z_Swing', aggfunc='sum'),
    O_Swing=pd.NamedAgg(column='O_Swing', aggfunc='sum'),
    Z_Contact=pd.NamedAgg(column='Z_Contact', aggfunc='sum'),
    O_Contact=pd.NamedAgg(column='O_Contact', aggfunc='sum'),
    Hard_Hit=pd.NamedAgg(column='Hard_Hit', aggfunc='sum')
)

In [372]:
team_batter_rates_p_throws = team_batter_totals_p_throws

team_batter_rates_p_throws['Pull_pct'] = team_batter_rates_p_throws['Pull']/team_batter_rates_p_throws['BIP_hit_direction']
team_batter_rates_p_throws['Cent_pct'] = team_batter_rates_p_throws['Cent']/team_batter_rates_p_throws['BIP_hit_direction']
team_batter_rates_p_throws['Oppo_pct'] = team_batter_rates_p_throws['Oppo']/team_batter_rates_p_throws['BIP_hit_direction']

team_batter_rates_p_throws['LD_pct'] = team_batter_rates_p_throws['LD']/team_batter_rates_p_throws['BIP_bb_type']
team_batter_rates_p_throws['GB_pct'] = team_batter_rates_p_throws['GB']/team_batter_rates_p_throws['BIP_bb_type']
team_batter_rates_p_throws['FB_pct'] = team_batter_rates_p_throws['FB']/team_batter_rates_p_throws['BIP_bb_type']

team_batter_rates_p_throws['Swing_pct'] = team_batter_rates_p_throws['Swing']/team_batter_rates_p_throws['Pitches']
team_batter_rates_p_throws['Contact_pct'] = team_batter_rates_p_throws['Contact']/team_batter_rates_p_throws['Swing']

team_batter_rates_p_throws['Z_Swing_pct'] = team_batter_rates_p_throws['Z_Swing']/team_batter_rates_p_throws['Zone']
team_batter_rates_p_throws['O_Swing_pct'] = team_batter_rates_p_throws['O_Swing']/team_batter_rates_p_throws['Zone']

team_batter_rates_p_throws['Z_Contact_pct'] = team_batter_rates_p_throws['Z_Contact']/team_batter_rates_p_throws['Z_Swing']
team_batter_rates_p_throws['O_Contact_pct'] = team_batter_rates_p_throws['O_Contact']/team_batter_rates_p_throws['O_Swing']

team_batter_rates_p_throws['Hard_Hit_pct'] = team_batter_rates_p_throws['Hard_Hit']/team_batter_rates_p_throws['BIP']

team_batter_rates_p_throws = team_batter_rates_p_throws[['BIP', 'BIP_hit_direction', 'Pull_pct', 'Cent_pct', 'Oppo_pct', 
                   'BIP_bb_type', 'LD_pct', 'GB_pct', 'FB_pct', 
                   'Pitches', 'Swing_pct', 'Contact_pct',
                   'Z_Swing_pct', 'O_Swing_pct', 'Z_Contact_pct', 'O_Contact_pct', 
                   'Hard_Hit_pct']]

team_batter_rates_p_throws.round(3)

BIP  BIP_hit_direction  Pull_pct  Cent_pct  \
batter             p_throws                                               
Aidan Muir         R           0                  0       NaN       NaN   
Brandon Latter     L           0                  0       NaN       NaN   
                   R           1                  1     0.000     0.000   
Brandon Nicholson  L           7                  3     0.333     0.667   
                   R          14                 14     0.357     0.500   
Brendon Daley      L           3                  3     0.667     0.000   
                   R           7                  7     0.429     0.286   
Bryan Rivera       L           1                  1     0.000     0.000   
                   R           4                  4     0.000     0.750   
Charlie Towers     L           1                  1     0.000     0.000   
                   R           9                  9     0.444     0.222   
Clayton Keyes      L           1                  1     0.000     0.000   
                   R           8                  8     0.250     0.250   
Connor Bowie       R           6                  6     0.500     0.167   
Danny Berg         L           5                  2     0.000     0.500   
                   R           9                  9     0.667     0.222   
Dennis Dei Baning  L           5                  3     0.333     0.000   
                   R          17                 17     0.353     0.235   
Dennis Dei Banning L           3                  3     0.667     0.000   
                   R           8                  8     0.375     0.625   
Jacob Underwood    L           3                  3     0.000     0.667   
                   R          12                 12     0.417     0.250   
Jordan Schulefand  R           0                  0       NaN       NaN   
Josh Niles         L           1                  1     0.000     0.000   
                   R          13                 13     0.692     0.231   
Keelor Loveridge   L           3                  0       NaN       NaN   
                   R           1                  0       NaN       NaN   
Laine Renaud       L           2                  0       NaN       NaN   
                   R          15                 14     0.429     0.143   
Luis Bernardo      L           9                  6     0.833     0.000   
                   R          30                 30     0.367     0.267   
Quin Cotton        L           2                  2     0.500     0.000   
                   R          11                 11     0.455     0.273   
Tanner Rempel      L           5                  5     0.600     0.400   
                   R          17                 16     0.500     0.188   
Tristan Osika      L           2                  2     0.000     0.500   
                   R           6                  6     0.667     0.333   
Troy Banks         R           2                  2     1.000     0.000   
Tyler Duncan       L           6                  4     0.500     0.250   
                   R          25                 25     0.280     0.320   
Zach Marriott      L           2                  0       NaN       NaN   
                   R           0                  0       NaN       NaN   

                             Oppo_pct  BIP_bb_type  LD_pct  GB_pct  FB_pct  \
batter             p_throws                                                  
Aidan Muir         R              NaN            0     NaN     NaN     NaN   
Brandon Latter     L              NaN            0     NaN     NaN     NaN   
                   R            1.000            1   1.000   0.000   0.000   
Brandon Nicholson  L            0.000            3   0.000   0.667   0.333   
                   R            0.143           14   0.214   0.357   0.429   
Brendon Daley      L            0.333            3   0.333   0.667   0.000   
                   R            0.286            7   0.286   0.571   0.143   
Bryan Rivera       L      

In [373]:
team_batter_rates_p_throws.loc[(batter, slice(None))].round(3)

,BIP,BIP_hit_direction,Pull_pct,Cent_pct,Oppo_pct,BIP_bb_type,LD_pct,GB_pct,FB_pct,Pitches,Swing_pct,Contact_pct,Z_Swing_pct,O_Swing_pct,Z_Contact_pct,O_Contact_pct,Hard_Hit_pct
p_throws,,,,,,,,,,,,,,,,,
L,9,6,0.833,0.000,0.167,6,0.167,0.500,0.333,29,0.655,0.737,0.889,1.222,0.750,0.727,0.222
R,30,30,0.367,0.267,0.367,30,0.133,0.333,0.533,109,0.505,0.855,0.815,1.222,0.909,0.818,0.400


In [374]:
team_batter_totals_pitch_category = team_batter_data.groupby(['batter', 'pitch_category']).agg(
    BIP=pd.NamedAgg(column='BIP', aggfunc='sum'),
    BIP_hit_direction=pd.NamedAgg(column='BIP_hit_direction', aggfunc='sum'),
    Pull=pd.NamedAgg(column='Pull', aggfunc='sum'),
    Cent=pd.NamedAgg(column='Cent', aggfunc='sum'),
    Oppo=pd.NamedAgg(column='Oppo', aggfunc='sum'),
    BIP_bb_type=pd.NamedAgg(column='BIP_bb_type', aggfunc='sum'),
    LD=pd.NamedAgg(column='LD', aggfunc='sum'),
    GB=pd.NamedAgg(column='GB', aggfunc='sum'),
    FB=pd.NamedAgg(column='FB', aggfunc='sum'),
    Pitches=pd.NamedAgg(column='Pitches', aggfunc='sum'),
    Zone=pd.NamedAgg(column='Zone', aggfunc='sum'),
    Swing=pd.NamedAgg(column='Swing', aggfunc='sum'),
    Contact=pd.NamedAgg(column='Contact', aggfunc='sum'),
    Z_Swing=pd.NamedAgg(column='Z_Swing', aggfunc='sum'),
    O_Swing=pd.NamedAgg(column='O_Swing', aggfunc='sum'),
    Z_Contact=pd.NamedAgg(column='Z_Contact', aggfunc='sum'),
    O_Contact=pd.NamedAgg(column='O_Contact', aggfunc='sum'),
    Hard_Hit=pd.NamedAgg(column='Hard_Hit', aggfunc='sum')
)

In [375]:
team_batter_rates_pitch_category = team_batter_totals_pitch_category

team_batter_rates_pitch_category['Pull_pct'] = team_batter_rates_pitch_category['Pull']/team_batter_rates_pitch_category['BIP_hit_direction']
team_batter_rates_pitch_category['Cent_pct'] = team_batter_rates_pitch_category['Cent']/team_batter_rates_pitch_category['BIP_hit_direction']
team_batter_rates_pitch_category['Oppo_pct'] = team_batter_rates_pitch_category['Oppo']/team_batter_rates_pitch_category['BIP_hit_direction']

team_batter_rates_pitch_category['LD_pct'] = team_batter_rates_pitch_category['LD']/team_batter_rates_pitch_category['BIP_bb_type']
team_batter_rates_pitch_category['GB_pct'] = team_batter_rates_pitch_category['GB']/team_batter_rates_pitch_category['BIP_bb_type']
team_batter_rates_pitch_category['FB_pct'] = team_batter_rates_pitch_category['FB']/team_batter_rates_pitch_category['BIP_bb_type']

team_batter_rates_pitch_category['Swing_pct'] = team_batter_rates_pitch_category['Swing']/team_batter_rates_pitch_category['Pitches']
team_batter_rates_pitch_category['Contact_pct'] = team_batter_rates_pitch_category['Contact']/team_batter_rates_pitch_category['Swing']

team_batter_rates_pitch_category['Z_Swing_pct'] = team_batter_rates_pitch_category['Z_Swing']/team_batter_rates_pitch_category['Zone']
team_batter_rates_pitch_category['O_Swing_pct'] = team_batter_rates_pitch_category['O_Swing']/team_batter_rates_pitch_category['Zone']

team_batter_rates_pitch_category['Z_Contact_pct'] = team_batter_rates_pitch_category['Z_Contact']/team_batter_rates_pitch_category['Z_Swing']
team_batter_rates_pitch_category['O_Contact_pct'] = team_batter_rates_pitch_category['O_Contact']/team_batter_rates_pitch_category['O_Swing']

team_batter_rates_pitch_category['Hard_Hit_pct'] = team_batter_rates_pitch_category['Hard_Hit']/team_batter_rates_pitch_category['BIP']

team_batter_rates_pitch_category = team_batter_rates_pitch_category[['BIP', 'BIP_hit_direction', 'Pull_pct', 'Cent_pct', 'Oppo_pct', 
                   'BIP_bb_type', 'LD_pct', 'GB_pct', 'FB_pct', 
                   'Pitches', 'Swing_pct', 'Contact_pct',
                   'Z_Swing_pct', 'O_Swing_pct', 'Z_Contact_pct', 'O_Contact_pct', 
                   'Hard_Hit_pct']]

team_batter_rates_pitch_category.round(3)

BIP  BIP_hit_direction  Pull_pct  Cent_pct  \
batter         pitch_category                                               
Aidan Muir     fastball          0                  0       NaN       NaN   
               offspeed          0                  0       NaN       NaN   
Brandon Latter fastball          1                  1     0.000     0.000   
               offspeed          0                  0       NaN       NaN   
               other             0                  0       NaN       NaN   
...                            ...                ...       ...       ...   
Tyler Duncan   breaking ball     8                  8     0.500     0.250   
               fastball         14                 14     0.143     0.357   
               offspeed          4                  4     0.500     0.250   
               other             5                  3     0.333     0.333   
Zach Marriott  other             2                  0       NaN       NaN   

                               Oppo_pct  BIP_bb_type  LD_pct  GB_pct  FB_pct  \
batter         pitch_category                                                  
Aidan Muir     fastball             NaN            0     NaN     NaN     NaN   
               offspeed             NaN            0     NaN     NaN     NaN   
Brandon Latter fastball           1.000            1   1.000   0.000   0.000   
               offspeed             NaN            0     NaN     NaN     NaN   
               other                NaN            0     NaN     NaN     NaN   
...                                 ...          ...     ...     ...     ...   
Tyler Duncan   breaking ball      0.250            8   0.000   0.875   0.125   
               fastball           0.500           14   0.071   0.429   0.500   
               offspeed           0.250            4   0.000   0.750   0.250   
               other              0.333            3   0.333   0.333   0.333   
Zach Marriott  other                NaN            0     NaN     NaN     NaN   

                               Pitches  Swing_pct  Contact_pct  Z_Swing_pct  \
batter         pitch_category                                                 
Aidan Muir     fastball              1      1.000        1.000          NaN   
               offspeed              2      1.000        0.000        1.000   
Brandon Latter fastball              4      0.250        1.000        0.500   
               offspeed              2      0.500        1.000        1.000   
               other                 0        NaN          NaN          NaN   
...                                ...        ...          ...          ...   
Tyler Duncan   breaking ball        23      0.609        0.929        0.833   
               fastball             68      0.603        0.829        0.952   
               offspeed             16      0.375        0.667        0.333   
               other                29      0.517        0.533          NaN   
Zach Marriott  other                12      0.417        0.800          NaN   

                               O_Swing_pct  Z_Contact_pct  O_Contact_pct  \
batter         pitch_category                                              
Aidan Muir     fastball                inf            NaN          1.000   
               offspeed              0.000           0.00            NaN   
Brandon Latter fastball              0.000           1.00            NaN   
               offspeed              0.000           1.00            NaN   
               other                   NaN            NaN            NaN   
...                                    ...            ...            ...   
Tyler Duncan   breaking ball         1.500           1.00          0.889   
               fastball              1.000           0.85          0.810   
               offspeed              1.667           1.00          0.600   
               other                   inf            NaN          0.533   
Zach Marriott  other                   inf      

In [376]:
team_batter_rates_pitch_category.loc[(batter, slice(None))].round(3)

,BIP,BIP_hit_direction,Pull_pct,Cent_pct,Oppo_pct,BIP_bb_type,LD_pct,GB_pct,FB_pct,Pitches,Swing_pct,Contact_pct,Z_Swing_pct,O_Swing_pct,Z_Contact_pct,O_Contact_pct,Hard_Hit_pct
pitch_category,,,,,,,,,,,,,,,,,
breaking ball,10,10,0.600,0.000,0.400,10,0.100,0.400,0.500,37,0.459,0.765,0.667,2.167,1.000,0.692,0.700
fastball,18,18,0.333,0.278,0.389,18,0.167,0.333,0.500,63,0.587,0.865,0.852,0.519,0.826,0.929,0.222
offspeed,3,3,1.000,0.000,0.000,3,0.333,0.333,0.333,9,0.444,1.000,1.000,0.333,1.000,1.000,0.333
other,8,5,0.200,0.600,0.200,5,0.000,0.400,0.600,29,0.552,0.750,NaN,inf,NaN,0.750,0.250


In [377]:
team_batter_totals_count = team_batter_data.groupby(['batter', 'count']).agg(
    BIP=pd.NamedAgg(column='BIP', aggfunc='sum'),
    BIP_hit_direction=pd.NamedAgg(column='BIP_hit_direction', aggfunc='sum'),
    Pull=pd.NamedAgg(column='Pull', aggfunc='sum'),
    Cent=pd.NamedAgg(column='Cent', aggfunc='sum'),
    Oppo=pd.NamedAgg(column='Oppo', aggfunc='sum'),
    BIP_bb_type=pd.NamedAgg(column='BIP_bb_type', aggfunc='sum'),
    LD=pd.NamedAgg(column='LD', aggfunc='sum'),
    GB=pd.NamedAgg(column='GB', aggfunc='sum'),
    FB=pd.NamedAgg(column='FB', aggfunc='sum'),
    Pitches=pd.NamedAgg(column='Pitches', aggfunc='sum'),
    Zone=pd.NamedAgg(column='Zone', aggfunc='sum'),
    Swing=pd.NamedAgg(column='Swing', aggfunc='sum'),
    Contact=pd.NamedAgg(column='Contact', aggfunc='sum'),
    Z_Swing=pd.NamedAgg(column='Z_Swing', aggfunc='sum'),
    O_Swing=pd.NamedAgg(column='O_Swing', aggfunc='sum'),
    Z_Contact=pd.NamedAgg(column='Z_Contact', aggfunc='sum'),
    O_Contact=pd.NamedAgg(column='O_Contact', aggfunc='sum'),
    Hard_Hit=pd.NamedAgg(column='Hard_Hit', aggfunc='sum')
)

In [378]:
team_batter_rates_count = team_batter_totals_count

team_batter_rates_count['Pull_pct'] = team_batter_rates_count['Pull']/team_batter_rates_count['BIP_hit_direction']
team_batter_rates_count['Cent_pct'] = team_batter_rates_count['Cent']/team_batter_rates_count['BIP_hit_direction']
team_batter_rates_count['Oppo_pct'] = team_batter_rates_count['Oppo']/team_batter_rates_count['BIP_hit_direction']

team_batter_rates_count['LD_pct'] = team_batter_rates_count['LD']/team_batter_rates_count['BIP_bb_type']
team_batter_rates_count['GB_pct'] = team_batter_rates_count['GB']/team_batter_rates_count['BIP_bb_type']
team_batter_rates_count['FB_pct'] = team_batter_rates_count['FB']/team_batter_rates_count['BIP_bb_type']

team_batter_rates_count['Swing_pct'] = team_batter_rates_count['Swing']/team_batter_rates_count['Pitches']
team_batter_rates_count['Contact_pct'] = team_batter_rates_count['Contact']/team_batter_rates_count['Swing']

team_batter_rates_count['Z_Swing_pct'] = team_batter_rates_count['Z_Swing']/team_batter_rates_count['Zone']
team_batter_rates_count['O_Swing_pct'] = team_batter_rates_count['O_Swing']/team_batter_rates_count['Zone']

team_batter_rates_count['Z_Contact_pct'] = team_batter_rates_count['Z_Contact']/team_batter_rates_count['Z_Swing']
team_batter_rates_count['O_Contact_pct'] = team_batter_rates_count['O_Contact']/team_batter_rates_count['O_Swing']

team_batter_rates_count['Hard_Hit_pct'] = team_batter_rates_count['Hard_Hit']/team_batter_rates_count['BIP']

team_batter_rates_count = team_batter_rates_count[['BIP', 'BIP_hit_direction', 'Pull_pct', 'Cent_pct', 'Oppo_pct', 
                   'BIP_bb_type', 'LD_pct', 'GB_pct', 'FB_pct', 
                   'Pitches', 'Swing_pct', 'Contact_pct',
                   'Z_Swing_pct', 'O_Swing_pct', 'Z_Contact_pct', 'O_Contact_pct', 
                   'Hard_Hit_pct']]

team_batter_rates_count.round(3)

BIP  BIP_hit_direction  Pull_pct  Cent_pct  Oppo_pct  \
batter         count                                                         
Aidan Muir     0-0      0                  0       NaN       NaN       NaN   
               0-1      0                  0       NaN       NaN       NaN   
               0-2      0                  0       NaN       NaN       NaN   
Brandon Latter 0-0      0                  0       NaN       NaN       NaN   
               1-0      0                  0       NaN       NaN       NaN   
...                   ...                ...       ...       ...       ...   
Zach Marriott  0-1      0                  0       NaN       NaN       NaN   
               0-2      0                  0       NaN       NaN       NaN   
               1-0      0                  0       NaN       NaN       NaN   
               1-1      1                  0       NaN       NaN       NaN   
               1-2      0                  0       NaN       NaN       NaN   

                      BIP_bb_type  LD_pct  GB_pct  FB_pct  Pitches  Swing_pct  \
batter         count                                                            
Aidan Muir     0-0              0     NaN     NaN     NaN        1        1.0   
               0-1              0     NaN     NaN     NaN        1        1.0   
               0-2              0     NaN     NaN     NaN        1        1.0   
Brandon Latter 0-0              0     NaN     NaN     NaN        1        0.0   
               1-0              0     NaN     NaN     NaN        1        0.0   
...                           ...     ...     ...     ...      ...        ...   
Zach Marriott  0-1              0     NaN     NaN     NaN        2        1.0   
               0-2              0     NaN     NaN     NaN        2        0.0   
               1-0              0     NaN     NaN     NaN        1        0.0   
               1-1              0     NaN     NaN     NaN        1        1.0   
               1-2              0     NaN     NaN     NaN        2        0.5   

                      Contact_pct  Z_Swing_pct  O_Swing_pct  Z_Contact_pct  \
batter         count                                                         
Aidan Muir     0-0            1.0          NaN          inf            NaN   
               0-1            0.0          1.0          0.0            0.0   
               0-2            0.0          1.0          0.0            0.0   
Brandon Latter 0-0            NaN          NaN          NaN            NaN   
               1-0            NaN          NaN          NaN            NaN   
...                           ...          ...          ...            ...   
Zach Marriott  0-1            1.0          NaN          inf            NaN   
               0-2            NaN          NaN          NaN            NaN   
               1-0            NaN          NaN          NaN            NaN   
               1-1            1.0          NaN          inf            NaN   
               1-2            0.0          NaN          inf            NaN   

                      O_Contact_pct  Hard_Hit_pct  
batter         count                               
Aidan Muir     0-0              1.0           NaN  
               0-1              NaN           NaN  
               0-2              NaN           NaN  
Brandon Latter 0-0              NaN           NaN  
               1-0              NaN           NaN  
...                             ...           ...  
Zach Marriott  0-1              1.0           NaN  
               0-2              NaN           NaN  
               1-0              NaN           NaN  
               1-1              1.0           0.0  
               1-2              0.0           NaN  

[220 rows x 17 columns]

In [382]:
team_batter_rates_count.loc[(batter, slice(None))].round(3)

,BIP,BIP_hit_direction,Pull_pct,Cent_pct,Oppo_pct,BIP_bb_type,LD_pct,GB_pct,FB_pct,Pitches,Swing_pct,Contact_pct,Z_Swing_pct,O_Swing_pct,Z_Contact_pct,O_Contact_pct,Hard_Hit_pct
count,,,,,,,,,,,,,,,,,
0-0,5,4,0.250,0.25,0.500,4,0.250,0.500,0.250,44,0.318,0.786,0.750,1.000,0.833,0.750,0.600
0-1,7,7,1.000,0.00,0.000,7,0.000,0.571,0.429,21,0.476,0.900,0.833,0.833,0.800,1.000,0.571
0-2,5,4,0.250,0.50,0.250,4,0.250,0.250,0.500,7,1.000,1.000,1.000,6.000,1.000,1.000,0.200
1-0,3,3,0.667,0.00,0.333,3,0.333,0.000,0.667,18,0.389,0.857,1.000,1.333,1.000,0.750,0.667
1-1,6,5,0.400,0.20,0.400,5,0.200,0.200,0.600,14,0.643,0.778,0.750,1.500,0.667,0.833,0.333
1-2,2,2,0.000,0.50,0.500,2,0.000,0.500,0.500,6,0.833,0.800,0.667,1.000,0.500,1.000,0.000
2-0,5,5,0.400,0.20,0.400,5,0.000,0.600,0.400,10,0.900,0.889,1.000,0.800,1.000,0.750,0.000
2-1,4,4,0.250,0.50,0.250,4,0.250,0.000,0.750,8,0.875,0.714,1.000,1.333,1.000,0.500,0.500
2-2,1,1,0.000,0.00,1.000,1,0.000,0.000,1.000,4,0.500,1.000,1.000,1.000,1.000,1.000,0.000
